In [57]:
# Install dependencies
!pip install -U funasr flask-ngrok flask

# Import libraries
import os
import json
import numpy as np
from flask import Flask, request, jsonify
from funasr import AutoModel


In [58]:
# Load model globally (only once)
model = AutoModel(model="emotion2vec_plus_large")
print("✅ Model loaded successfully!")


funasr version: 1.2.7.
Check update of funasr, and it would cost few times. You may disable it by set `disable_update=True` in AutoModel
You are using the latest version of funasr-1.2.7


Warning, miss key in ckpt: modality_encoders.AUDIO.decoder.blocks.0.0.weight, /root/.cache/modelscope/hub/models/iic/emotion2vec_plus_large/model.pt
Warning, miss key in ckpt: modality_encoders.AUDIO.decoder.blocks.0.0.bias, /root/.cache/modelscope/hub/models/iic/emotion2vec_plus_large/model.pt
Warning, miss key in ckpt: modality_encoders.AUDIO.decoder.blocks.1.0.weight, /root/.cache/modelscope/hub/models/iic/emotion2vec_plus_large/model.pt
Warning, miss key in ckpt: modality_encoders.AUDIO.decoder.blocks.1.0.bias, /root/.cache/modelscope/hub/models/iic/emotion2vec_plus_large/model.pt
Warning, miss key in ckpt: modality_encoders.AUDIO.decoder.blocks.2.0.weight, /root/.cache/modelscope/hub/models/iic/emotion2vec_plus_large/model.pt
Warning, miss key in ckpt: modality_encoders.AUDIO.decoder.blocks.2.0.bias, /root/.cache/modelscope/hub/models/iic/emotion2vec_plus_large/model.pt
Warning, miss key in ckpt: modality_encoders.AUDIO.decoder.blocks.3.0.weight, /root/.cache/modelscope/hub/models

In [59]:
app = Flask(__name__)

@app.route("/predict", methods=["POST"])
def predict():
    try:
        file = request.files["audio"]
        file_path = "temp.wav"
        file.save(file_path)

        # Run model
        result = model.generate(file_path, granularity="utterance")
        data = result[0]

        if 'predictions' in data and 'scores' in data:
            emotions = data['predictions']
            scores = data['scores']
        elif 'labels' in data and 'scores' in data:
            emotions = [e.split('/')[-1] for e in data['labels']]
            scores = data['scores']
        else:
            return jsonify({"error": "No 'labels' or 'predictions' found"}), 500

        best_idx = int(np.argmax(scores))
        best_emotion = emotions[best_idx]
        confidence = float(scores[best_idx])
        top_emotions = sorted(zip(emotions, scores), key=lambda x: x[1], reverse=True)
        top_emotions_json = [{"emotion": e, "score": float(s)} for e, s in top_emotions]

        return jsonify({
            "emotion": best_emotion,
            "confidence": confidence,
            "topEmotions": top_emotions_json
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500


In [ ]:
# Install pyngrok
!pip install pyngrok
from pyngrok import ngrok

# Set ngrok authtoken
# Replace "YOUR_AUTH_TOKEN" with your actual ngrok authentication token
ngrok.set_auth_token("31y3EqKzzZqEmFcksJJiO0jFShJ_76ywiqbQqegsSHLULmtL")

# Start tunnel
public_url = ngrok.connect(5001)
print("🔗 Public URL:", public_url)

# Run Flask app
app.run(port=5001)

🔗 Public URL: NgrokTunnel: "https://6ac77767e787.ngrok-free.app" -> "http://localhost:5001"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5001
INFO:werkzeug:Press CTRL+C to quit
